In [1]:
import requests
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# htmlの文字化けを解消したい。

from bs4 import BeautifulSoup
import chardet

def normalize_html_content(content):
    # bytesならエンコーディング推定→デコード
    if isinstance(content, (bytes, bytearray)):
        enc = chardet.detect(content).get("encoding") or "utf-8"
        text = content.decode(enc, errors="replace")
    else:
        text = content

    # <meta charset="utf-8"> を保証
    soup = BeautifulSoup(text, "html.parser")
    if soup.head is None:
        # head がない場合は作る
        html = soup.new_tag("html")
        head = soup.new_tag("head")
        body = soup.new_tag("body")
        body.append(soup)  # 既存ノードを body 下へ
        html.append(head)
        html.append(body)
        soup = BeautifulSoup(str(html), "html.parser")

    if not soup.head.find("meta", attrs={"charset": True}):
        meta = soup.new_tag("meta", charset="utf-8")
        soup.head.insert(0, meta)
    else:
        soup.head.find("meta", attrs={"charset": True})["charset"] = "utf-8"

    # テーブルの線を表示するためのCSSスタイルを追加
    style = soup.new_tag("style")
    style.string = """
    table {
        border-collapse: collapse;
        width: 100%;
        margin: 10px 0;
    }
    table, th, td {
        border: 1px solid #000;
    }
    th, td {
        padding: 8px;
        text-align: left;
    }
    th {
        background-color: #f2f2f2;
        font-weight: bold;
    }
    """
    soup.head.append(style)

    return str(soup)

In [3]:
def save_html(html, pdf_path, output_dir):
    output_path = output_dir / pdf_path.with_suffix(".html").name
    html = normalize_html_content(html)
    with open(output_path, "w") as f:
        f.write(html)

def save_markdown(markdown, pdf_path, output_dir):
    output_path = output_dir / pdf_path.with_suffix(".md").name
    with open(output_path, "w") as f:
        f.write(markdown)

## Upstage/Document Parse

In [4]:
def run_upstage(pdf_path, output_dir: Path = Path("../output/upstage"), type: str = "html", save: bool = True):
    url = "https://api.upstage.ai/v1/document-digitization"
    api_key = os.getenv("UPSTAGE_API_KEY")
    headers = {"Authorization": f"Bearer {api_key}"}
    # extracted_pdf_pathを使用（前のセルで設定された一時PDFファイル）
    files = {"document": open(pdf_path, "rb")}
    data = {"ocr": "auto", "model": "document-parse-nightly"}
    response = requests.post(url, headers=headers, files=files, data=data)
    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code} - {response.text}")
    if type == "html":
        content = response.json()["content"]["html"]
    elif type == "markdown":
        content = response.json()["content"]["markdown"]
    if save:
        output_path = output_dir / pdf_path.parent.name
        output_path.mkdir(parents=True, exist_ok=True)
        if type == "html":
            save_html(content, pdf_path, output_path)
        elif type == "markdown":
            save_markdown(content, pdf_path, output_path)
    return content

## Azure Document Intelligence

In [5]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, AnalyzeDocumentRequest, DocumentContentFormat

client = DocumentIntelligenceClient(
            endpoint=os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT"),
            credential=AzureKeyCredential(os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY"))
        )

def run_azure_di(pdf_path, output_dir: Path = Path("../output/azure"), save: bool = True):
    with pdf_path.open("rb") as file:
        poller = client.begin_analyze_document(
            model_id="prebuilt-layout",
            body=file,
            output_content_format=DocumentContentFormat.MARKDOWN
        )
    result: AnalyzeResult = poller.result()
    if save:
        output_path = output_dir / pdf_path.parent.name
        output_path.mkdir(parents=True, exist_ok=True)
        save_markdown(result.content, pdf_path, output_path)
    return result.content

## YOMITOKU

In [6]:
import cv2
import nest_asyncio

from yomitoku import DocumentAnalyzer
from yomitoku.data.functions import load_pdf, load_image

# Jupyter環境でasyncioを使用するための設定
nest_asyncio.apply()



def run_yomitoku(pdf_path: Path, output_dir: Path = Path("../output/yomitoku"), save: bool = True):

    analyzer = DocumentAnalyzer(visualize=True, device="cpu")

    # PDFファイルを読み込み

    imgs = load_pdf(pdf_path)
    output_results = []
    for i, img in enumerate(imgs):

        results, ocr_vis, layout_vis = analyzer(img)

        # HTML形式で解析結果をエクスポート
        parent_path = output_dir / pdf_path.parent.name 
        parent_path.mkdir(parents=True, exist_ok=True)
        output_path = parent_path / (pdf_path.name.split(".")[0] + f"_{i}.html")
        results.to_html(str(output_path), img=img)
        output_results.append(results)
        # 可視化画像を保存
        output_ocr_path = parent_path / (pdf_path.name.split(".")[0] + f"_ocr_{i}.jpg")
        cv2.imwrite(str(output_ocr_path), ocr_vis)
        output_layout_path = parent_path / (pdf_path.name.split(".")[0] + f"_layout_{i}.jpg")
        cv2.imwrite(str(output_layout_path), layout_vis)
    return output_results

## Gemini 2.5 flash

In [7]:
from google import genai
from google.genai import types
from pdf2image import convert_from_path
import os
import io
from dotenv import load_dotenv

load_dotenv()


def run_gemini(pdf_path: Path, output_dir: Path = Path("../output/gemini"), save: bool = True):
    # 全ページを画像に変換
    images = convert_from_path(pdf_path, dpi=200)
    
    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
    
    PROMPT = """
    これは PDF ドキュメントのページです。構造を維持しながら、すべてのテキストコンテンツを抽出してください。
    テーブル、列、見出し、および構造化されたコンテンツに特に注意を払ってください。

    テーブルの場合：
    1. マークダウンテーブル形式を使用してテーブル構造を維持
    2. すべての列ヘッダーと行ラベルを保持
    3. 数値データが正確にキャプチャされていることを確認

    マルチカラムレイアウトの場合：
    1. 左から右へ列を処理
    2. 異なる列のコンテンツを明確に区別

    チャートやグラフの場合：
    1. チャートのタイプを説明
    2. 可視の軸ラベル、凡例、データポイントを抽出
    3. タイトルやキャプションを抽出

    イラストの場合：
    1. イラストのタイトルを抽出
    2. イラストの内容を説明

    段落の区切りと書式を維持してください。
    すべての見出し、フッター、ページ番号、脚注を維持してください。

    出力は必ずマークダウン形式で行って下さい。
    """
    
    # 各ページを処理して結果を格納
    page_outputs = []
    
    for i, image in enumerate(images):
        # 画像をバイト形式に変換
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='JPEG')
        image_bytes = img_byte_arr.getvalue()
        
        # 各ページを個別に処理
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=[
                types.Part.from_bytes(
                    data=image_bytes,
                    mime_type='image/jpeg',
                ),
                PROMPT + f"\n\n（これはページ {i+1}/{len(images)} です）"
            ]
        )
        
        page_outputs.append(f"<!-- ページ {i+1} -->\n{response.text}")
    
    # 全ページの結果を結合
    output = "\n\n---\n\n".join(page_outputs)
    
    if save:
        output_path = output_dir / pdf_path.parent.name
        output_path.mkdir(parents=True, exist_ok=True)
        save_markdown(output, pdf_path, output_path)
    
    return output

In [8]:
import time
import json
from datetime import datetime

def measure_time(func, *args, **kwargs):
    """関数の実行時間を計測するデコレータ"""
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    execution_time = end_time - start_time
    return result, execution_time

def save_timing_results(timing_data, output_dir="timing_results"):
    """タイミング結果をJSONファイルに保存"""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"timing_results_{timestamp}.json"
    filepath = output_path / filename
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(timing_data, f, ensure_ascii=False, indent=2)
    
    print(f"タイミング結果を保存しました: {filepath}")
    return filepath


In [9]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor, AutoModelForImageTextToText
from qwen_vl_utils import process_vision_info
import torch
import fitz  # PyMuPDF
from PIL import Image
import io

# モデルキャッシュ用のグローバル変数
_models_cache = {}

def download_models():
    """Qwen2.5VLとQwen3VLのモデルを事前にダウンロードしてキャッシュする"""
    
    print("Qwen2.5VLモデルをダウンロード中...")
    device, dtype = _select_device_and_dtype()
    
    # Qwen2.5VLモデルのダウンロード
    qwen25vl_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2.5-VL-7B-Instruct",
        dtype=dtype,
        torch_dtype=dtype,
        device_map="auto" if device.type != "cpu" else None
    )
    qwen25vl_processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
    
    _models_cache['qwen25vl'] = {
        'model': qwen25vl_model,
        'processor': qwen25vl_processor,
        'device': device,
        'dtype': dtype
    }
    
    print("Qwen3VLモデルをダウンロード中...")
    
    # Qwen3VLモデルのダウンロード
    qwen3vl_model = AutoModelForImageTextToText.from_pretrained(
        "Qwen/Qwen3-VL-235B-A22B-Instruct", 
        dtype=dtype,
        torch_dtype=dtype,
        device_map="auto" if device.type != "cpu" else None
    )
    qwen3vl_processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-235B-A22B-Instruct")
    
    _models_cache['qwen3vl'] = {
        'model': qwen3vl_model,
        'processor': qwen3vl_processor,
        'device': device,
        'dtype': dtype
    }
    
    print("モデルのダウンロードが完了しました！")

def _select_device_and_dtype():
    """デバイスとデータ型を選択する"""
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        dtype = torch.float16
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        dtype = torch.float16
    else:
        device = torch.device("cpu")
        dtype = torch.float32
    return device, dtype

## Qwen2.5-VL/Qwen3-VL

In [10]:
# 共通のプロンプトを定義
PROMPT = """
これは PDF ドキュメントのページです。構造を維持しながら、すべてのテキストコンテンツを抽出してください。
テーブル、列、見出し、および構造化されたコンテンツに特に注意を払ってください。

テーブルの場合：
1. マークダウンテーブル形式を使用してテーブル構造を維持
2. すべての列ヘッダーと行ラベルを保持
3. 数値データが正確にキャプチャされていることを確認

マルチカラムレイアウトの場合：
1. 左から右へ列を処理
2. 異なる列のコンテンツを明確に区別

チャートやグラフの場合：
1. チャートのタイプを説明
2. 可視の軸ラベル、凡例、データポイントを抽出
3. タイトルやキャプションを抽出

イラストの場合：
1. イラストのタイトルを抽出
2. イラストの内容を説明

段落の区切りと書式を維持してください。
すべての見出し、フッター、ページ番号、脚注を維持してください。

出力は必ずマークダウン形式で行って下さい。
"""

def process_single_page_qwen(model_info, image, page_num, total_pages, prompt=PROMPT):
    """単一ページを処理する共通関数（最適化済み）"""

    model = model_info['model']
    processor = model_info['processor']
    device = model_info['device']
    
    # メッセージを構築
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": prompt + f"\n\n（これはページ {page_num + 1}/{total_pages} です）"}
            ]
        }
    ]

    # プロンプトの準備
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    
    # バッチ処理用の入力準備
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = {k: v.to(device) if hasattr(v, "to") else v for k, v in inputs.items()}

    # 推論実行（最適化されたパラメータ）
    with torch.no_grad():  # メモリ使用量を削減
        generated_ids = model.generate(
            **inputs, 
            max_new_tokens=2048,
            do_sample=False,  # 確定的な出力
            temperature=0.1,  # 低い温度で一貫性を向上
            pad_token_id=processor.tokenizer.eos_token_id,
            use_cache=True  # KVキャッシュを使用
        )
    
    generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs["input_ids"], generated_ids)]
    output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    return f"<!-- ページ {page_num + 1} -->\n{output_text[0]}"


In [11]:
def run_qwen25vl_optimized(pdf_path, output_dir=None, save=False):
    """
    Qwen2.5-VLを使用してPDFからテキストを抽出する関数（最適化版）
    事前にダウンロードされたモデルを使用
    """
   
    # モデルがダウンロードされていない場合はダウンロード
    if 'qwen25vl' not in _models_cache:
        print("Qwen2.5VLモデルがダウンロードされていません。ダウンロードを開始します...")
        download_models()
    
    model_info = _models_cache['qwen25vl']
    
    # PDFを開く
    doc = fitz.open(pdf_path)
    total_pages = len(doc)
    
    # 各ページを個別に処理
    page_outputs = []
    
    for page_num in range(total_pages):
        page = doc[page_num]
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # 2倍解像度
        img_data = pix.tobytes("png")
        image = Image.open(io.BytesIO(img_data))
        
        # 最適化されたページ処理
        page_output = process_single_page_qwen(model_info, image, page_num, total_pages)
        page_outputs.append(page_output)
    
    # 全ページの結果を結合
    response_content = "\n\n---\n\n".join(page_outputs)

    if save and output_dir is not None:
        output_path = output_dir / pdf_path.parent.name
        output_path.mkdir(parents=True, exist_ok=True)
        save_markdown(response_content, pdf_path, output_path)

    doc.close()
    return response_content

def run_qwen3vl_optimized(pdf_path, output_dir=None, save=False):
    """
    Qwen3-VLを使用してPDFを処理する関数（最適化版）
    事前にダウンロードされたモデルを使用
    """
    import fitz  # PyMuPDF
    from PIL import Image
    import io
    
    # モデルがダウンロードされていない場合はダウンロード
    if 'qwen3vl' not in _models_cache:
        print("Qwen3VLモデルがダウンロードされていません。ダウンロードを開始します...")
        download_models()
    
    model_info = _models_cache['qwen3vl']
    
    # PDFを開く
    doc = fitz.open(pdf_path)
    total_pages = len(doc)

    # 各ページを個別に処理
    page_outputs = []
    
    for page_num in range(total_pages):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # 2倍解像度
        img_data = pix.tobytes("png")
        image = Image.open(io.BytesIO(img_data))
        
        # 最適化されたページ処理
        page_output = process_single_page_qwen(model_info, image, page_num, total_pages)
        page_outputs.append(page_output)
    
    # 全ページの結果を結合
    response_content = "\n\n---\n\n".join(page_outputs)

    if save and output_dir is not None:
        output_path = output_dir / pdf_path.parent.name
        output_path.mkdir(parents=True, exist_ok=True)
        save_markdown(response_content, pdf_path, output_path)

    doc.close()
    return response_content


In [12]:
# 使用方法の例
def initialize_models():
    """モデルを初期化する関数（最初に1回だけ実行）"""
    print("モデルを初期化中...")
    download_models()
    print("初期化完了！")

def clear_model_cache():
    """モデルキャッシュをクリアする関数（メモリ不足時など）"""
    global _models_cache
    _models_cache.clear()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    print("モデルキャッシュをクリアしました")

# 実行速度向上のための追加の最適化関数
def optimize_for_speed():
    """実行速度向上のための設定"""
    
    # PyTorchの最適化設定
    torch.backends.cudnn.benchmark = True  # CUDA最適化
    torch.backends.cudnn.deterministic = False  # 速度優先
    
    # メモリ効率化
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print("速度最適化設定を適用しました")

def get_model_info():
    """現在のモデルキャッシュの状態を確認"""
    print("=== モデルキャッシュ状態 ===")
    for model_name, info in _models_cache.items():
        print(f"{model_name}: 読み込み済み")
        print(f"  デバイス: {info['device']}")
        print(f"  データ型: {info['dtype']}")
    if not _models_cache:
        print("モデルは読み込まれていません")


In [13]:
def print_timing_summary(timing_data):
    """タイミング結果のサマリーを表示"""
    print("\n" + "="*60)
    print("実行時間サマリー")
    print("="*60)
    
    # 各モデルの平均実行時間を計算
    model_stats = {}
    
    for result in timing_data["results"]:
        for model_name, model_data in result["models"].items():
            if model_name not in model_stats:
                model_stats[model_name] = {
                    "total_time": 0,
                    "success_count": 0,
                    "error_count": 0,
                    "times": []
                }
            
            if model_data["status"] == "success":
                model_stats[model_name]["total_time"] += model_data["execution_time"]
                model_stats[model_name]["success_count"] += 1
                model_stats[model_name]["times"].append(model_data["execution_time"])
            else:
                model_stats[model_name]["error_count"] += 1
    
    # 結果を表示
    for model_name, stats in model_stats.items():
        print(f"\n{model_name}:")
        print(f"  成功: {stats['success_count']} ファイル")
        print(f"  エラー: {stats['error_count']} ファイル")
        
        if stats["success_count"] > 0:
            avg_time = stats["total_time"] / stats["success_count"]
            min_time = min(stats["times"])
            max_time = max(stats["times"])
            print(f"  平均実行時間: {avg_time:.2f} 秒")
            print(f"  最短実行時間: {min_time:.2f} 秒")
            print(f"  最長実行時間: {max_time:.2f} 秒")
            print(f"  総実行時間: {stats['total_time']:.2f} 秒")
    
    print("\n" + "="*60)


In [14]:
def run_baseline(file_list:list[Path]):
    for file_path in file_list:
        print(f"Processing {file_path}...")
        run_upstage(file_path, output_dir=Path("../output/upstage"), save=True)
        print("Processing Upstage/Document Parse... done")
        run_azure_di(file_path, output_dir=Path("../output/azure"), save=True)
        print("Processing Azure/Document Intelligence... done")
        run_yomitoku(file_path, output_dir=Path("../output/yomitoku"), save=True)
        print("Processing YOMITOKU... done")
        run_gemini(file_path, output_dir=Path("../output/gemini"), save=True)
        print("Processing Gemini 2.5 Flash... done")

def run_qwen(file_list:list[Path]):
    initialize_models()
    for file_path in file_list:
        run_qwen25vl_optimized(file_path, output_dir=Path("../output/qwen25vl"), save=True)
        run_qwen3vl_optimized(file_path, output_dir=Path("../output/qwen3vl"), save=True)


In [15]:
# accelerateエラーを修正したdownload_models関数
def download_models_fixed():
    """Qwen2.5VLとQwen3VLのモデルを事前にダウンロードしてキャッシュする（accelerateエラー修正版）"""
    import torch
    
    print("Qwen2.5VLモデルをダウンロード中...")
    device, dtype = _select_device_and_dtype()
    
    # accelerateが利用可能かチェック
    try:
        import accelerate
        use_device_map = True
        print("accelerateが利用可能です。device_mapを使用します。")
    except ImportError:
        use_device_map = False
        print("accelerateが利用できません。device_mapを使用せずにモデルを読み込みます。")
    
    # Qwen2.5VLモデルのダウンロード
    try:
        if use_device_map and device.type != "cpu":
            qwen25vl_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
                "Qwen/Qwen2.5-VL-7B-Instruct",
                dtype=dtype,
                device_map="auto"
            )
        else:
            qwen25vl_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
                "Qwen/Qwen2.5-VL-7B-Instruct",
                dtype=dtype
            )
            qwen25vl_model = qwen25vl_model.to(device)
        
        qwen25vl_processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
        
        _models_cache['qwen25vl'] = {
            'model': qwen25vl_model,
            'processor': qwen25vl_processor,
            'device': device,
            'dtype': dtype
        }
        print("Qwen2.5VLモデルのダウンロード完了")
        
    except Exception as e:
        print(f"Qwen2.5VLモデルのダウンロードに失敗しました: {e}")
        raise
    
    print("Qwen3VLモデルをダウンロード中...")
    
    # Qwen3VLモデルのダウンロード
    # try:
    #     if use_device_map and device.type != "cpu":
    #         qwen3vl_model = AutoModelForImageTextToText.from_pretrained(
    #             "Qwen/Qwen3-VL-235B-A22B-Instruct", 
    #             torch_dtype=dtype,
    #             device_map="auto"
    #         )
    #     else:
    #         qwen3vl_model = AutoModelForImageTextToText.from_pretrained(
    #             "Qwen/Qwen3-VL-235B-A22B-Instruct", 
    #             torch_dtype=dtype
    #         )
    #         qwen3vl_model = qwen3vl_model.to(device)
        
    #     qwen3vl_processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-235B-A22B-Instruct")
        
    #     _models_cache['qwen3vl'] = {
    #         'model': qwen3vl_model,
    #         'processor': qwen3vl_processor,
    #         'device': device,
    #         'dtype': dtype
    #     }
    #     print("Qwen3VLモデルのダウンロード完了")
        
    # except Exception as e:
    #     print(f"Qwen3VLモデルのダウンロードに失敗しました: {e}")
    #     raise
    
    print("モデルのダウンロードが完了しました！")

# 修正版のinitialize_models関数
def initialize_models_fixed():
    """モデルを初期化する関数（最初に1回だけ実行）- accelerateエラー修正版"""
    print("モデルを初期化中...")
    download_models_fixed()
    print("初期化完了！")


In [16]:
# 修正版のrun_qwen関数
def run_qwen_fixed(file_list:list[Path]):
    """Qwenモデル処理を実行（accelerateエラー修正版）"""
    initialize_models_fixed()
    for file_path in file_list:
        print(f"Processing {file_path}...")
        run_qwen25vl_optimized(file_path, output_dir=Path("../output/qwen25vl"), save=True)
        run_qwen3vl_optimized(file_path, output_dir=Path("../output/qwen3vl"), save=True)

# 修正版のrun_qwen_timed関数
def run_qwen_timed_fixed(file_list:list[Path]):
    """Qwenモデル処理を実行し、各モデルの計算時間を計測（accelerateエラー修正版）"""
    timing_data = {
        "timestamp": datetime.now().isoformat(),
        "total_files": len(file_list),
        "results": []
    }
    
    # モデルを初期化
    print("モデルを初期化中...")
    initialize_models_fixed()
    
    for file_idx, file_path in enumerate(file_list):
        print(f"Processing {file_path}... ({file_idx + 1}/{len(file_list)})")
        
        file_result = {
            "file_path": str(file_path),
            "file_name": file_path.name,
            "models": {}
        }
        
        # Qwen2.5VL
        print("  Processing Qwen2.5VL...")
        try:
            _, exec_time = measure_time(run_qwen25vl_optimized, file_path, output_dir=Path("../output/qwen25vl"), save=True)
            file_result["models"]["qwen25vl"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Qwen2.5VL completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["qwen25vl"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Qwen2.5VL failed: {e}")
        
        # Qwen3VL
        print("  Processing Qwen3VL...")
        try:
            _, exec_time = measure_time(run_qwen3vl_optimized, file_path, output_dir=Path("../output/qwen3vl"), save=True)
            file_result["models"]["qwen3vl"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Qwen3VL completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["qwen3vl"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Qwen3VL failed: {e}")
        
        timing_data["results"].append(file_result)
        print(f"  File {file_idx + 1} completed\n")
    
    # タイミング結果を保存
    save_timing_results(timing_data, "timing_results_qwen")
    
    # サマリーを表示
    print_timing_summary(timing_data)
    
    return timing_data


In [17]:
# 日時ベースの出力フォルダ構造に対応した修正版関数

def run_baseline_timed_with_datetime(file_list:list[Path]):
    """ベースライン処理を実行し、各モデルの計算時間を計測（日時フォルダ対応版）"""
    from datetime import datetime
    
    # 日時ベースの出力フォルダを作成
    timestamp = datetime.now().strftime("%Y%m%d-%H%M")
    base_output_dir = Path(f"../output/{timestamp}")
    base_output_dir.mkdir(parents=True, exist_ok=True)
    
    timing_data = {
        "timestamp": datetime.now().isoformat(),
        "total_files": len(file_list),
        "output_base_dir": str(base_output_dir),
        "results": []
    }
    
    for file_idx, file_path in enumerate(file_list):
        print(f"Processing {file_path}... ({file_idx + 1}/{len(file_list)})")
        
        file_result = {
            "file_path": str(file_path),
            "file_name": file_path.name,
            "models": {}
        }
        
        # Upstage/Document Parse
        print("  Processing Upstage/Document Parse...")
        try:
            _, exec_time = measure_time(run_upstage, file_path, output_dir=base_output_dir / "upstage", save=True)
            file_result["models"]["upstage"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Upstage completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["upstage"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Upstage failed: {e}")
        
        # Azure Document Intelligence
        print("  Processing Azure/Document Intelligence...")
        try:
            _, exec_time = measure_time(run_azure_di, file_path, output_dir=base_output_dir / "azure", save=True)
            file_result["models"]["azure"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Azure completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["azure"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Azure failed: {e}")
        
        # YOMITOKU
        print("  Processing YOMITOKU...")
        try:
            _, exec_time = measure_time(run_yomitoku, file_path, output_dir=base_output_dir / "yomitoku", save=True)
            file_result["models"]["yomitoku"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    YOMITOKU completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["yomitoku"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    YOMITOKU failed: {e}")
        
        # Gemini 2.5 Flash
        print("  Processing Gemini 2.5 Flash...")
        try:
            _, exec_time = measure_time(run_gemini, file_path, output_dir=base_output_dir / "gemini", save=True)
            file_result["models"]["gemini"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Gemini completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["gemini"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Gemini failed: {e}")
        
        timing_data["results"].append(file_result)
        print(f"  File {file_idx + 1} completed\n")
    
    # タイミング結果を保存（outputフォルダ内に保存）
    save_timing_results(timing_data, str(base_output_dir / "timing_results"))
    
    # サマリーを表示
    print_timing_summary(timing_data)
    
    print(f"\n出力フォルダ: {base_output_dir}")
    return timing_data

def run_qwen_timed_with_datetime(file_list:list[Path]):
    """Qwenモデル処理を実行し、各モデルの計算時間を計測（日時フォルダ対応版）"""
    from datetime import datetime
    
    # 日時ベースの出力フォルダを作成
    timestamp = datetime.now().strftime("%Y%m%d-%H%M")
    base_output_dir = Path(f"../output/{timestamp}")
    base_output_dir.mkdir(parents=True, exist_ok=True)
    
    timing_data = {
        "timestamp": datetime.now().isoformat(),
        "total_files": len(file_list),
        "output_base_dir": str(base_output_dir),
        "results": []
    }
    
    # モデルを初期化
    print("モデルを初期化中...")
    initialize_models_fixed()
    
    for file_idx, file_path in enumerate(file_list):
        print(f"Processing {file_path}... ({file_idx + 1}/{len(file_list)})")
        
        file_result = {
            "file_path": str(file_path),
            "file_name": file_path.name,
            "models": {}
        }
        
        # Qwen2.5VL
        print("  Processing Qwen2.5VL...")
        try:
            _, exec_time = measure_time(run_qwen25vl_optimized, file_path, output_dir=base_output_dir / "qwen25vl", save=True)
            file_result["models"]["qwen25vl"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Qwen2.5VL completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["qwen25vl"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Qwen2.5VL failed: {e}")
        
        # Qwen3VL
        # print("  Processing Qwen3VL...")
        # try:
        #     _, exec_time = measure_time(run_qwen3vl_optimized, file_path, output_dir=base_output_dir / "qwen3vl", save=True)
        #     file_result["models"]["qwen3vl"] = {
        #         "status": "success",
        #         "execution_time": exec_time
        #     }
        #     print(f"    Qwen3VL completed in {exec_time:.2f} seconds")
        # except Exception as e:
        #     file_result["models"]["qwen3vl"] = {
        #         "status": "error",
        #         "error": str(e),
        #         "execution_time": 0
        #     }
        #     print(f"    Qwen3VL failed: {e}")
        
        timing_data["results"].append(file_result)
        print(f"  File {file_idx + 1} completed\n")
    
    # タイミング結果を保存（outputフォルダ内に保存）
    save_timing_results(timing_data, str(base_output_dir / "timing_results"))
    
    # サマリーを表示
    print_timing_summary(timing_data)
    
    print(f"\n出力フォルダ: {base_output_dir}")
    return timing_data


In [18]:
# 使用例とフォルダ構造の説明
print("修正版関数の使用方法:")
print("="*50)
print("1. ベースライン処理（日時フォルダ対応）:")
print("   baseline_results = run_baseline_timed_with_datetime(files_list)")
print()
print("2. Qwenモデル処理（日時フォルダ対応）:")
print("   qwen_results = run_qwen_timed_with_datetime(files_list)")
print()
print("出力フォルダ構造:")
print("output/")
print("├── 20250126-1430/          # 実行日時フォルダ")
print("│   ├── upstage/            # Upstage/Document Parse結果")
print("│   ├── azure/              # Azure Document Intelligence結果")
print("│   ├── yomitoku/           # YOMITOKU結果")
print("│   ├── gemini/             # Gemini 2.5 Flash結果")
print("│   ├── qwen25vl/           # Qwen2.5VL結果")
print("│   ├── qwen3vl/            # Qwen3VL結果")
print("│   └── timing_results/     # タイミング結果")
print("│       └── timing_results_20250126_143000.json")
print("└── 20250126-1500/          # 別の実行時のフォルダ")
print("    ├── upstage/")
print("    ├── azure/")
print("    ├── ...")
print("    └── timing_results/")
print("        └── timing_results_20250126_150000.json")


修正版関数の使用方法:
1. ベースライン処理（日時フォルダ対応）:
   baseline_results = run_baseline_timed_with_datetime(files_list)

2. Qwenモデル処理（日時フォルダ対応）:
   qwen_results = run_qwen_timed_with_datetime(files_list)

出力フォルダ構造:
output/
├── 20250126-1430/          # 実行日時フォルダ
│   ├── upstage/            # Upstage/Document Parse結果
│   ├── azure/              # Azure Document Intelligence結果
│   ├── yomitoku/           # YOMITOKU結果
│   ├── gemini/             # Gemini 2.5 Flash結果
│   ├── qwen25vl/           # Qwen2.5VL結果
│   ├── qwen3vl/            # Qwen3VL結果
│   └── timing_results/     # タイミング結果
│       └── timing_results_20250126_143000.json
└── 20250126-1500/          # 別の実行時のフォルダ
    ├── upstage/
    ├── azure/
    ├── ...
    └── timing_results/
        └── timing_results_20250126_150000.json


In [19]:
# 新しいrun_baseline関数（タイミング計測付き）
def run_baseline_timed(file_list:list[Path]):
    """ベースライン処理を実行し、各モデルの計算時間を計測"""
    timing_data = {
        "timestamp": datetime.now().isoformat(),
        "total_files": len(file_list),
        "results": []
    }
    
    for file_idx, file_path in enumerate(file_list):
        print(f"Processing {file_path}... ({file_idx + 1}/{len(file_list)})")
        
        file_result = {
            "file_path": str(file_path),
            "file_name": file_path.name,
            "models": {}
        }
        
        # Upstage/Document Parse
        print("  Processing Upstage/Document Parse...")
        try:
            _, exec_time = measure_time(run_upstage, file_path, output_dir=Path("../output/upstage"), save=True)
            file_result["models"]["upstage"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Upstage completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["upstage"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Upstage failed: {e}")
        
        # Azure Document Intelligence
        print("  Processing Azure/Document Intelligence...")
        try:
            _, exec_time = measure_time(run_azure_di, file_path, output_dir=Path("../output/azure"), save=True)
            file_result["models"]["azure"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Azure completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["azure"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Azure failed: {e}")
        
        # YOMITOKU
        print("  Processing YOMITOKU...")
        try:
            _, exec_time = measure_time(run_yomitoku, file_path, output_dir=Path("../output/yomitoku"), save=True)
            file_result["models"]["yomitoku"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    YOMITOKU completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["yomitoku"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    YOMITOKU failed: {e}")
        
        # Gemini 2.5 Flash
        print("  Processing Gemini 2.5 Flash...")
        try:
            _, exec_time = measure_time(run_gemini, file_path, output_dir=Path("../output/gemini"), save=True)
            file_result["models"]["gemini"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Gemini completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["gemini"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Gemini failed: {e}")
        
        timing_data["results"].append(file_result)
        print(f"  File {file_idx + 1} completed\n")
    
    # タイミング結果を保存
    save_timing_results(timing_data)
    
    # サマリーを表示
    print_timing_summary(timing_data)
    
    return timing_data

# 新しいrun_qwen関数（タイミング計測付き）
def run_qwen_timed(file_list:list[Path]):
    """Qwenモデル処理を実行し、各モデルの計算時間を計測"""
    timing_data = {
        "timestamp": datetime.now().isoformat(),
        "total_files": len(file_list),
        "results": []
    }
    
    # モデルを初期化
    print("モデルを初期化中...")
    initialize_models()
    
    for file_idx, file_path in enumerate(file_list):
        print(f"Processing {file_path}... ({file_idx + 1}/{len(file_list)})")
        
        file_result = {
            "file_path": str(file_path),
            "file_name": file_path.name,
            "models": {}
        }
        
        # Qwen2.5VL
        print("  Processing Qwen2.5VL...")
        try:
            _, exec_time = measure_time(run_qwen25vl_optimized, file_path, output_dir=Path("../output/qwen25vl"), save=True)
            file_result["models"]["qwen25vl"] = {
                "status": "success",
                "execution_time": exec_time
            }
            print(f"    Qwen2.5VL completed in {exec_time:.2f} seconds")
        except Exception as e:
            file_result["models"]["qwen25vl"] = {
                "status": "error",
                "error": str(e),
                "execution_time": 0
            }
            print(f"    Qwen2.5VL failed: {e}")
        
        # Qwen3VL
        # print("  Processing Qwen3VL...")
        # try:
        #     _, exec_time = measure_time(run_qwen3vl_optimized, file_path, output_dir=Path("../output/qwen3vl"), save=True)
        #     file_result["models"]["qwen3vl"] = {
        #         "status": "success",
        #         "execution_time": exec_time
        #     }
        #     print(f"    Qwen3VL completed in {exec_time:.2f} seconds")
        # except Exception as e:
        #     file_result["models"]["qwen3vl"] = {
        #         "status": "error",
        #         "error": str(e),
        #         "execution_time": 0
        #     }
        #     print(f"    Qwen3VL failed: {e}")
        
        timing_data["results"].append(file_result)
        print(f"  File {file_idx + 1} completed\n")
    
    # タイミング結果を保存
    save_timing_results(timing_data, "timing_results_qwen")
    
    # サマリーを表示
    print_timing_summary(timing_data)
    
    return timing_data


### 最適化されたQwenモデルの使用方法

#### 1. モデルの初期化（最初に1回だけ実行）
```python
# モデルを事前にダウンロードしてキャッシュ
initialize_models()
```

#### 2. 最適化された関数の使用
```python
# 最適化されたQwen2.5VL
content_qwen25vl = run_qwen25vl_optimized(file_path, output_dir=Path("../output/qwen25vl"), save=True)

# 最適化されたQwen3VL  
content_qwen3vl = run_qwen3vl_optimized(file_path, output_dir=Path("../output/qwen3vl"), save=True)
```

#### 3. その他の便利な関数
```python
# モデルキャッシュの状態確認
get_model_info()

# 速度最適化設定の適用
optimize_for_speed()

# メモリ不足時のキャッシュクリア
clear_model_cache()
```

## タイミング計測機能の使用方法

### 1. ベースラインモデルの実行（タイミング計測付き）
```python
# タイミング計測付きでベースラインモデルを実行
timing_results = run_baseline_timed(files_list)
```

### 2. Qwenモデルの実行（タイミング計測付き）
```python
# タイミング計測付きでQwenモデルを実行
timing_results = run_qwen_timed(files_list)
```

### 3. 保存される情報
- **JSONファイル**: `timing_results/` ディレクトリに保存
- **各PDFファイルごと**:
  - ファイル名とパス
  - 各モデルの実行時間
  - 成功/エラー状態
  - エラーメッセージ（エラー時）
- **サマリー情報**:
  - 平均実行時間
  - 最短/最長実行時間
  - 成功/エラー件数
  - 総実行時間

### 4. 実行例
```python
# ベースラインモデルを実行
baseline_results = run_baseline_timed(files_list[:2])  # 最初の2ファイルでテスト

# Qwenモデルを実行
qwen_results = run_qwen_timed(files_list[:2])  # 最初の2ファイルでテスト
```


## 実行プロセス

In [20]:
base_dir = Path("../data/")
files_list = [f for f in base_dir.glob("**/*.pdf")]
files_list

[PosixPath('../data/tables/04_要件定義書_表の中の表.pdf'),
 PosixPath('../data/tables/03_要件定義書_各種テーブル.pdf'),
 PosixPath('../data/tegaki/20250915_業務日報.pdf'),
 PosixPath('../data/tegaki/0686240_ツミタス汎用_表.pdf'),
 PosixPath('../data/tegaki/0708742_JBEC・KEC_ミライト.pdf'),
 PosixPath('../data/others/縦書き_ルビあり.pdf'),
 PosixPath('../data/receipt/納入済通知書送付票_カスレ_低解像度.pdf'),
 PosixPath('../data/receipt/領収書_歯科_sample.pdf'),
 PosixPath('../data/receipt/01_invoice_sample_01.pdf'),
 PosixPath('../data/receipt/02_receipt_sample.pdf'),
 PosixPath('../data/thesis/202303_ExDistilBERT_東芝.pdf'),
 PosixPath('../data/graphs/08_グラフ表混在_portlait_日本経済見通し_JRI_202507.pdf'),
 PosixPath('../data/graphs/07_グラフ表混在_landscape_日本経済展望_JRI_202507.pdf'),
 PosixPath('../data/insurance/tokyo_marine_yakkan_sample.pdf'),
 PosixPath('../data/insurance/nissei_miraino_katachi_sample.pdf')]

In [21]:
# baseline_results = run_baseline_timed(files_list)

In [ ]:
qwen_results = run_qwen_timed_with_datetime(files_list)

モデルを初期化中...
モデルを初期化中...
Qwen2.5VLモデルをダウンロード中...
accelerateが利用可能です。device_mapを使用します。


Loading checkpoint shards: 100%|██████████| 5/5 [00:06<00:00,  1.26s/it]
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


Qwen2.5VLモデルのダウンロード完了
Qwen3VLモデルをダウンロード中...
モデルのダウンロードが完了しました！
初期化完了！
Processing ../data/tables/04_要件定義書_表の中の表.pdf... (1/15)
  Processing Qwen2.5VL...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    Qwen2.5VL completed in 354.15 seconds
  File 1 completed

Processing ../data/tables/03_要件定義書_各種テーブル.pdf... (2/15)
  Processing Qwen2.5VL...
    Qwen2.5VL completed in 526.37 seconds
  File 2 completed

Processing ../data/tegaki/20250915_業務日報.pdf... (3/15)
  Processing Qwen2.5VL...
    Qwen2.5VL completed in 40.44 seconds
  File 3 completed

Processing ../data/tegaki/0686240_ツミタス汎用_表.pdf... (4/15)
  Processing Qwen2.5VL...
    Qwen2.5VL completed in 107.12 seconds
  File 4 completed

Processing ../data/tegaki/0708742_JBEC・KEC_ミライト.pdf... (5/15)
  Processing Qwen2.5VL...
    Qwen2.5VL completed in 253.92 seconds
  File 5 completed

Processing ../data/others/縦書き_ルビあり.pdf... (6/15)
  Processing Qwen2.5VL...
    Qwen2.5VL completed in 252.36 seconds
  File 6 completed

Processing ../data/receipt/納入済通知書送付票_カスレ_低解像度.pdf... (7/15)
  Processing Qwen2.5VL...
    Qwen2.5VL completed in 253.70 seconds
  File 7 completed

Processing ../data/receipt/領収書_歯科_sample.pdf... (8/15)
  Processing Qwen